In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import metrics, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
METRICS = [metrics.RootMeanSquaredError(name='rms'), metrics.MeanAbsoluteError(name='mae')]

In [ ]:
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 50

In [ ]:
def make_and_train_autoencoder(X_train, metrics=METRICS):
    
    len_input_output = X_train.shape[-1]
    input_ = Input(shape=(len_input_output,))
    encoded = Dense(units=ENCODING_DIM*2, activation="relu")(input_)
    bottleneck = Dense(units=ENCODING_DIM, 
                       activation="relu")(encoded)
    decoded = Dense(units=ENCODING_DIM*2, 
                    activation="relu")(bottleneck)
    output = Dense(units=len_input_output, 
                    activation="linear")(decoded)
    #Training is performed on the entire autoencoder
    autoencoder = Model(inputs=input_, outputs=output)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error',
                        metrics=[metrics])
    autoencoder.fit(X_train, X_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)
    #Use only the encoder part for dimensionality reduction
    encoder = Model(inputs=input_, outputs=bottleneck)
    return autoencoder, encoder

In [ ]:
df = pd.read_csv('normalised_clean_data.csv')
X, y = df.iloc[:,:-1], df.iloc[:, -1]




In [ ]:
for i in [4,8,16]:
  ENCODING_DIM = i
  auto, encode = make_and_train_autoencoder(X, metrics=METRICS)
  q = encode.predict(X)
  w = np.array(q)
  df = pd.DataFrame(data=w,index=None,columns=None)
  df['label'] = y
  name = "data" + str( i)
  df.to_csv(name, index=False)

Epoch 1/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0187 - rms: 0.1005 - mae: 0.0497
Epoch 2/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0072 - rms: 0.0846 - mae: 0.0515
Epoch 3/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0056 - rms: 0.0748 - mae: 0.0450
Epoch 4/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0055 - rms: 0.0743 - mae: 0.0450
Epoch 5/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0055 - rms: 0.0741 - mae: 0.0452
Epoch 6/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0055 - rms: 0.0739 - mae: 0.0452
Epoch 7/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0054 - rms: 0.0736 - mae: 0.0453
Epoch 8/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0053 - rms: 0.0731 - mae: 0.0451
Epoch 9/50
469/469 [==============================] - 1s 2ms/step - loss: 0.0052 - rms: 0.0719 - mae: 0.0445
Epoch 10/50
469/469

In [ ]:
df = pd.read_csv('data16.csv')

In [ ]:
df_train, df_test_valid = train_test_split( df ,stratify=df['label'], test_size=0.3)
df_test, df_validation = train_test_split( df_test_valid, stratify=df_test_valid['label'], test_size=0.5)

In [ ]:
df_train.to_csv('training_set_autoencoded.csv', index=False)
df_validation.to_csv('validation_set_autoencoded.csv', index=False)
df_test.to_csv('test_set_autoencoded.csv', index=False)